In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `C:\Users\nsthtz\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [2]:
using DelimitedFiles
#data = readdlm(joinpath("dummyMajor.csv"),';')
#data = readdlm(joinpath("dummyMajorA#.csv"),';')
data = readdlm(joinpath("dummyMajorMultiple.csv"),';')

#data = readdlm(joinpath("test.csv"),';')

#data = readdlm(joinpath("dummyMinor.csv"),',')
#data = readdlm(joinpath("dummyMinorA#.csv"),';')
data = round.(Int, data)

128×24 Array{Int64,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮         

# # Convert piano roll into timestepx12 tonal mapping


In [228]:
function getActiveNotes(timestep)
    return findall(x->x==1, data[:,timestep])
end

tonal_map = zeros(Int8, size(data, 2), 12)
i=1
data[:, 1]

for i in 1:size(data, 2)
    active_notes = getActiveNotes(i)
    for note in active_notes
        tonal_map[i, note%12+1] += 1
    end
end

tonal_map



24×12 Array{Int8,2}:
 1  1  1  0  1  0  0  0  0  0  0  0
 0  0  1  0  0  1  0  0  1  1  0  0
 1  0  0  0  2  0  0  0  1  0  0  0
 0  0  0  0  0  2  0  0  0  1  1  0
 1  0  0  0  0  1  0  1  0  0  0  0
 1  0  0  1  0  0  0  0  0  2  0  0
 0  0  0  1  0  0  0  0  0  0  0  1
 1  0  0  0  0  1  0  0  0  0  0  0
 1  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  1  0  0  0  1  0  0  0  1
 1  0  1  0  0  0  0  0  0  1  0  0
 1  0  0  0  0  0  0  1  0  0  1  0
 0  0  0  0  0  2  0  0  0  0  0  0
 0  0  1  0  1  0  0  0  0  1  0  0
 0  0  1  0  0  0  1  0  1  0  0  0
 2  0  1  0  0  0  0  0  2  0  0  0
 0  0  1  0  0  1  0  0  0  0  0  1
 1  0  0  1  0  0  0  0  0  0  0  0
 0  1  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  0  1
 1  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  1  0  0
 0  0  0  0  1  0  0  0  0  0  0  1
 1  1  0  0  0  0  0  0  0  0  0  0

# # Pattern matcher to identify the best-fit key and scale from existing notes

In [4]:
# Functions and Global

pattern = [1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1]

# Weight of core notes (1, 4, 5) doubled
majorpattern = [1, 0, 0.5, 0, 0.5, 1, 0, 1, 0, 0.5, 0, 0.5]
minorpattern = [0.5, 0, 1, 0, 1, 0.5, 0, 0.5, 0, 1, 0, 0.5]

Keys = Dict{Integer, String}(1 => "C", 2 => "C#", 3 => "D", 4 => "D#", 5 => "E", 6 => "F", 7 => "F#", 8 => "G", 9 => "G#", 10 => "A", 11 => "A#", 12 => "B")


#Score In: Tx12 Matrix, pattern. Out: Best position(s)
function score_key(matrix, pattern)
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    println("Key Counter is ", counter)
    
    key_score = zeros(Int8, 12, 1)
    
    for i in 1:size(pattern, 1)
        for j in 1:size(pattern, 1)
            key_score[i] += circshift(pattern, i-1)[j]*counter[j]
        end
    end
    println("Key Score is ", key_score)
    return key_score
end

# Find maximum from list
function findmaximum(list)
    positions = []
    max_value = maximum(list)
    for (i, x) in enumerate(list)
        if x == max_value
            push!(positions, i)
        end
    end
    return positions
end

# Find an optimal scale
function score_tonality(matrix, positions)
    
    counter = zeros(Int8, 1, 12)
    for i in 1:size(matrix, 2)
        counter[i] = sum(matrix[:,i])
    end
    
    tonality_scores = []
    
    for position in positions
        counter_shifted = circshift(counter, (0,-(position-1)))
        println("Position Counter for ", position, " is ", counter_shifted)
        
        major_score = 0
        minor_score = 0
        
        for i in 1:size(counter_shifted, 2)
            major_score += majorpattern[i]*counter_shifted[i]
            minor_score += minorpattern[i]*counter_shifted[i]
        end

        if major_score > minor_score
            push!(tonality_scores, [position, major_score, 1])
        else
            push!(tonality_scores, [position, minor_score, 0])
        end
        
    end 
    
    return tonality_scores
end

score_tonality (generic function with 1 method)

In [5]:
# Score matrix after pattern
key_score_result = score_key(tonal_map, pattern)
# score_result = [9, 5, 18, 9, 23, 18, 8, 2, 7, 16, 12, 11]

# Find positions of maximum score
positions = findmaximum(key_score_result)
sort!(positions)

# # Use positions to find the best key

tonality_results = score_tonality(tonal_map, positions)

# [Position (1 = C), Score, Scale (1 = Major, 0 = Minor)]
best_key = [0, 0, 1]

# In case of tied scores, check each for the best scale
for result_group in tonality_results
    if best_key[2] < result_group[2]
        best_key[1] = result_group[1]
        best_key[2] = round.(Int, result_group[2])
        best_key[3] = result_group[3]
    end
end
    
#Save Key
println(best_key)
if best_key[3] == 1
    key = best_key[1]
    key_print = Keys[best_key[1]]
    scale = "Major"
else
    key = (12+best_key[1]-3)%12
    key_print = Keys[key]
    scale = "Minor"
end
    
println(key, " ", key_print, " ", scale)

Key Counter is Int8[13 3 7 4 5 10 2 5 5 7 2 5]
Key Score is Int8[52; 39; 34; 46; 31; 49; 31; 44; 42; 34; 48; 26]
Position Counter for 1 is Int8[13 3 7 4 5 10 2 5 5 7 2 5]
[1, 40, 1]
1 C Major


# # Define patterns for chord functions relative to key

In [6]:
# Function entry: "Name" => ([notes], [Bass note], [Double/Triple notes], [Rhythmical Weight (Strong, Weak, Off)])
functions = Dict{String, Array{Int64, 1}}

if scale == "Major"
#Major  =  [1, 0, 2, 0, 3, 4, 0, 5, 0, 6, 0, 7]
#
    
    functions = Dict(
    "T"    =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O", "A"]),
    "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O", "D"]),
    "T-5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/3"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((4+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O"]),
    "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/5"  =>  (circshift([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
    "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D64"  =>  (circshift([4, 0, 0, 0, 6, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 0, 6, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O", "D"]),
    "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((11+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O"]),
    "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
    "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O"]),
    "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 0, 3], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O"]),
    "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 0, 3], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S", "W", "O"]),
    "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Ss"   =>  (circshift([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 1) ], [ (((2+(key-1))%12)+1, 1), (((5+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
    "Ts"   =>  (circshift([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], (key-1)), [ (((9+(key-1))%12)+1, 1) ], [ (key, 3), (((9+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Tm"   =>  (circshift([0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 5], (key-1)), [ (((4+(key-1))%12)+1, 1) ], [ (((4+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))

elseif scale == "Minor"
#                         [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,11]
#Minor  =                 [1, 0, 2, 3, 0, 4, 0, 5, 6, 0, 7, 7+]              
    functions = Dict(
    "T"    =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "S"    =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D"    =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T-5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0], (key-1)), [ (key, 1) ], [ (key, 1) ], ["S", "W", "O"]),
    "D-5"  =>  (circshift([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/3"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((3+(key-1))%12)+1, 3) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["S", "W", "O"]),
    "S/3"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 3) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "D/3"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((10+(key-1))%12)+1, 3) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "T/5"  =>  (circshift([1, 0, 0, 3, 0, 0, 0, 5, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 5) ], [ (key, 1), (((7+(key-1))%12)+1, 5) ], ["W", "O"]),
    "S/5"  =>  (circshift([5, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 5) ], [ (key, 5), (((5+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D/5"  =>  (circshift([0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 3, 0], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((7+(key-1))%12)+1, 1) ], ["W", "O"]),
    "D64"  =>  (circshift([4, 0, 0, 6, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "T64"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 0, 6, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "T54"  =>  (circshift([1, 0, 0, 0, 0, 4, 0, 5, 0, 0, 0, 0], (key-1)), [ (((0+(key-1))%12)+1, 1) ], [ (key, 1) ], ["S"]),
    "D54"  =>  (circshift([4, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S"]),
    "D7"   =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "D7/3" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((10+(key-1))%12)+1, 3) ], [ nothing ], ["S", "W", "O"]),
    "D7/5" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ nothing ], ["W", "O"]),
    "D7/7" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((5+(key-1))%12)+1, 7) ], [ nothing ], ["S", "W", "O"]),
    "D7-1" =>  (circshift([0, 0, 5, 0, 0, 7, 0, 0, 0, 0, 3, 0], (key-1)), [ (((2+(key-1))%12)+1, 5) ], [ (((2+(key-1))%12)+1, 5), (((5+(key-1))%12)+1, 7) ], ["S", "W", "O"]),
    "D7-5" =>  (circshift([0, 0, 0, 0, 0, 7, 0, 1, 0, 0, 3, 0], (key-1)), [ (((7+(key-1))%12)+1, 1) ], [ (((7+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "S65"  =>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ nothing ], ["S", "W", "O"]),
    "S65/6"=>  (circshift([5, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((2+(key-1))%12)+1, 6) ], [ nothing ], ["S"]),
    "S6"   =>  (circshift([0, 0, 6, 0, 0, 1, 0, 0, 3, 0, 0, 0], (key-1)), [ (((5+(key-1))%12)+1, 1) ], [ (((5+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Dm"   =>  (circshift([0, 0, 3, 0, 0, 5, 0, 0, 0, 0, 1, 0], (key-1)), [ (((10+(key-1))%12)+1, 1) ], [ (((10+(key-1))%12)+1, 1), (((2+(key-1))%12)+1, 3) ], ["S", "W", "O"]),
    "Ts"   =>  (circshift([3, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0, 0], (key-1)), [ (((8+(key-1))%12)+1, 1) ], [ (key, 3), (((8+(key-1))%12)+1, 1) ], ["S", "W", "O"]),
    "Tm"   =>  (circshift([0, 0, 0, 1, 0, 0, 0, 3, 0, 0, 5, 0], (key-1)), [ (((3+(key-1))%12)+1, 1) ], [ (((3+(key-1))%12)+1, 1), (((7+(key-1))%12)+1, 3) ], ["S", "W", "O"]))
       
end
      
for k in functions
    println(k[1], " ", k[2])
end

#      [C, C#, D, D#, E, F, F#, G, G#, A, A#, B]
#      [1, 2,  3, 4,  5, 6, 7,  8, 9, 10, 11, 12]
#      [2, 0,  3, 4,  0, 5, 0,  6, 0, 7,  1,  0]

S65/6 ([5, 0, 6, 0, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(3, 6)], Nothing[nothing], ["S", "W", "O"])
S/5 ([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(1, 5)], Tuple{Int64,Int64}[(1, 5), (6, 1)], ["W", "O"])
T ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(1, 1)], Tuple{Int64,Int64}[(1, 1)], ["S", "W", "O", "A"])
Ts ([3, 0, 0, 0, 5, 0, 0, 0, 0, 1, 0, 0], Tuple{Int64,Int64}[(10, 1)], Tuple{Int64,Int64}[(1, 3), (10, 1)], ["S", "W", "O"])
S/3 ([5, 0, 0, 0, 0, 1, 0, 0, 0, 3, 0, 0], Tuple{Int64,Int64}[(10, 3)], Tuple{Int64,Int64}[(1, 5), (6, 1)], ["S", "W", "O"])
T/5 ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64}[(8, 5)], Tuple{Int64,Int64}[(1, 1), (8, 5)], ["W", "O"])
Ss ([0, 0, 1, 0, 0, 3, 0, 0, 0, 5, 0, 0], Tuple{Int64,Int64}[(3, 1)], Tuple{Int64,Int64}[(3, 1), (6, 3)], ["S", "W", "O"])
D7/5 ([0, 0, 5, 0, 0, 7, 0, 1, 0, 0, 0, 3], Tuple{Int64,Int64}[(3, 5)], Nothing[nothing], ["W", "O"])
T/3 ([1, 0, 0, 0, 3, 0, 0, 5, 0, 0, 0, 0], Tuple{Int64,Int64

# # Analyze tonal map and calculate hamming distances for each timestep

In [223]:
##### DISTANCE FUNCTIONS NOT USED

function hamming(input, target)
    counter = 0
    for i in 1:size(input, 1)
        if input[i] != target[i]
            counter = counter+1
        end
    end
    return counter
end

## What is the distance to transform an input into a function NOT USED
    
function distance(input, target)
    distance = 0
    target_padded = copy(target)
    target_padded = append!(append!(target_padded, target), target)[1:36]
    for i in 1:size(input, 1)
        if input[i] > 0
            if target[i] > 0
                for x in 0:2
                    target_padded[i+(12*x)] = 0
                end
            else
                for j in 1:6
                    if target_padded[12+i-j] > 0
                        distance += j
                        break
                    elseif target_padded[12+i+j] > 0
                        distance += j
                        break
                    end
                end
            end
        end
    end
    
    return distance
end

## What is the distance of finding a valid function within the input NOT USED
        
function distance(input, target)
    distance = 0
    input_padded = append!(append!(input, input), input)[1:36]
    for i in 1:size(target, 1)
        if target[i] > 0
            if input[i] == 0
                for j in 1:6
                    if input_padded[12+i-j] > 0
                        distance += j
                        break
                    elseif input_padded[12+i+j] > 0
                        distance += j
                        break
                    end
                end
            end
        end
    end
                
    return distance
end
        

##### TODO: Adjust distance function so that it counts cumulative distance from one note in target to the nearest in input, Left-Left matching, remove upon match                

function tonal_distance(input, target)
    input = copy(input)
    distance = 0
    input_padded = append!(append!(input, input), input)[1:36]
    for i in 1:size(target, 1)
        if target[i] > 0
            if input_padded[12+i] == 0
                for j in 1:6
                    if input_padded[12+i-j] > 0
                        distance += j
                        for k in 1:2
                            input_padded[k*12+i-j] -= 1
                        end 
                        break
                    elseif input_padded[12+i+j] > 0
                        distance += j
                        for k in 0:1
                            input_padded[k*12+i+j] -= 1
                        end
                        break
                    end
                end
            elseif input_padded[12+i] > 0
                for k in 0:2
                    input_padded[k*12+i] -= 1
                end
            end
        end
    end
                
    return distance
end       

chords = keys(functions)
function_potential = Array{Tuple{Int64,String}}[]


for i in 1:size(tonal_map, 1)         
    function_pool = Tuple{Int64, String}[]
    for chord in chords
        tonal_dist = tonal_distance(tonal_map[i,:], functions[chord][1])
        if chord == "D-5" || chord == "T-5"
            tonal_dist += 1
        end
                            
                            
        function_pool_entry = (tonal_dist, chord)
        push!(function_pool, (tonal_dist, chord))
        

    end
    
    push!(function_potential, sort(function_pool))
end

display(function_potential[1])

## Naive function label
                    
chosen_chords = []                    
                  
    
for timestep in 1:size(function_potential, 1)
    append!(chosen_chords, [function_potential[timestep][1][2]])
end
                    
println(chosen_chords)


27-element Array{Tuple{Int64,String},1}:
 (1, "T-5")  
 (2, "D7-1") 
 (3, "D54")  
 (4, "D")    
 (4, "D/3")  
 (4, "D/5")  
 (4, "S6")   
 (4, "Ss")   
 (4, "Ts")   
 (5, "D-5")  
 (5, "D64")  
 (5, "S")    
 (5, "S/3")  
 ⋮           
 (5, "S65/6")
 (5, "T")    
 (5, "T/3")  
 (5, "T/5")  
 (5, "T64")  
 (6, "T54")  
 (6, "Tm")   
 (7, "D7-5") 
 (8, "D7")   
 (8, "D7/3") 
 (8, "D7/5") 
 (8, "D7/7") 

Any[0, 4, 7]
Any[1, 5, 8]
60
Any["T-5", "S6", "D64", "S", "T54", "Ts", "D54", "S", "D", "D", "D54", "D-5", "D7-5", "S6", "D7", "D54", "D7-1", "D54", "D7", "D-5", "S", "D7-5", "D64", "D54"]


# # Construct the search space for note distribution to voices

In [72]:
using Combinatorics

# Finds and returns all possible distributions of the valid note classes for a given function

function getVoicingPermutations(func)
    voicing_permutations = Array{Tuple{Int64,Int64}, 1}[]
    voicings_initial = []
    
    function_map = functions[func][1]
    double_note = functions[func][3]
    bass_note = functions[func][2][1]

    active_notes = sort!([(i, x) for (i, x) in enumerate(function_map) if x > 0], by = x -> x[2])   # [position, value]
    
    filter!(e->e != bass_note, active_notes)
        
    voicing_sans_bass = copy(active_notes)
            
    if size(active_notes, 1) == 1
        push!(voicing_sans_bass, bass_note)
        push!(voicing_sans_bass, bass_note)
        push!(voicings_initial, voicing_sans_bass)
    elseif size(active_notes, 1) == 2
        for i in 1:size(double_note, 1)
            voicing_sans_bass = copy(active_notes)
            push!(voicing_sans_bass, double_note[i])
            push!(voicings_initial, voicing_sans_bass)
        end
    elseif size(active_notes, 1) == 3
        push!(voicings_initial, voicing_sans_bass)
    end
                    
    for entry in voicings_initial
        voicings = collect(permutations(entry))
        
        for j in voicings
            push!(voicing_permutations, j)
        end
    end
    
    voicing_permutations = unique(voicing_permutations)   
    for p in voicing_permutations
        pushfirst!(p, bass_note)
    end
    
    return voicing_permutations
end
        
# Generates the candidate solutions of note distribution for a given function
               
function generateCandidates(func)
    voicing_permutations = getVoicingPermutations(func)
    valid_voicings = Array{Tuple{Int64,Int64},1}[]
    note_repository = []   # [[all C's], [all C#'s], [all D's], ..., [all B's]]
    for note_class in 1:12
        push!(note_repository, [x for x in 40:80 if x%12 == (note_class-1)])
    end

    
    
    for permutation in voicing_permutations
        #println(permutation)
        voicing = []
        for b in note_repository[permutation[1][1]]
            if b in 40:60
                for t in note_repository[permutation[2][1]]
                    if t in 47:68 && t >= b && t-b <= 20
                        for a in note_repository[permutation[3][1]]
                            if a in 52:72 && a >= t && a-t <= 12                                 
                                for s in note_repository[permutation[4][1]]
                                    if s in 59:80 && s >= a && s-a <= 12  
                                        voicing = Tuple{Int64,Int64}[]
                                        push!(voicing, (b, permutation[1][2]), (t, permutation[2][2]), (a, permutation[3][2]), (s, permutation[4][2])) 
                                        push!(valid_voicings, voicing)
                                    end                                
                                end           
                            end    
                        end
                        
                    end
                end       
            end
        end
    end
    
    ## Enforce Special rules
    
    if (func == "T-5") || (func == "D-5")
        placeholder = Array{Tuple{Int64,Int64},1}[]
        for voicing in valid_voicings
            if (voicing[4][2] == 1) && (voicing[4][1] - voicing[3][1] == 8)
                push!(placeholder, voicing)
            end
        end
        valid_voicings = placeholder
    end
    #println(typeof(valid_voicings))
    return(valid_voicings)
end

function_permutations = Dict{String, Array{Any, 1}}()

for keys in keys(functions)
    entry = unique(generateCandidates(keys))
    function_permutations[keys] = unique(generateCandidates(keys))
end

display(function_permutations["T"])

15-element Array{Any,1}:
 Tuple{Int64,Int64}[(48, 1), (52, 3), (55, 5), (60, 1)]
 Tuple{Int64,Int64}[(48, 1), (64, 3), (67, 5), (72, 1)]
 Tuple{Int64,Int64}[(60, 1), (64, 3), (67, 5), (72, 1)]
 Tuple{Int64,Int64}[(48, 1), (52, 3), (60, 1), (67, 5)]
 Tuple{Int64,Int64}[(48, 1), (64, 3), (72, 1), (79, 5)]
 Tuple{Int64,Int64}[(60, 1), (64, 3), (72, 1), (79, 5)]
 Tuple{Int64,Int64}[(48, 1), (55, 5), (64, 3), (72, 1)]
 Tuple{Int64,Int64}[(48, 1), (55, 5), (60, 1), (64, 3)]
 Tuple{Int64,Int64}[(48, 1), (67, 5), (72, 1), (76, 3)]
 Tuple{Int64,Int64}[(60, 1), (67, 5), (72, 1), (76, 3)]
 Tuple{Int64,Int64}[(48, 1), (60, 1), (64, 3), (67, 5)]
 Tuple{Int64,Int64}[(60, 1), (60, 1), (64, 3), (67, 5)]
 Tuple{Int64,Int64}[(48, 1), (48, 1), (55, 5), (64, 3)]
 Tuple{Int64,Int64}[(48, 1), (60, 1), (67, 5), (76, 3)]
 Tuple{Int64,Int64}[(60, 1), (60, 1), (67, 5), (76, 3)]

In [21]:
#INITIALIZATIONS

Tonics = ["T", "T/3", "T/5", "T-5"]
Tonics_dependant = ["T64", "T54"]
Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
Dominants = ["D", "D/3", "D/5", "D-5", "D64", "D54", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
Mediants = ["Ts", "Tm", "Ss", "Dm"]
beat_accentuation = Dict("O" => 1, "W" => 2, "S" => 3)
dominant_order = Dict("D" => 3, "D/3" => 3, "D/5" => 3, "D-5" => 3, "D64" => 1, "D54" => 2, "D7" => 4, "D7-1" => 4, "D7-5" => 4, "D7/3" => 4, "D7/5" => 4, "D7/7" => 4)

time_signature = (4, 2)

(4, 2)

# # Rhythm function

In [77]:
function rhythmify(time_signature, original) # (beats per bar, subdivisions) (3,1), (3, 2), (4,1), (4, 2)
    
    rhythm_map = String[]
    
    beat_remainder = size(original, 2) % (time_signature[1] * time_signature[2])
    
    rhythms = Dict(
        (3, 1) => ["S", "W", "W"],
        (3, 2) => ["S", "O", "W", "O", "W", "O"],
        (4, 1) => ["S", "W", "S", "W"],
        (4, 2) => ["S", "O", "W", "O", "S", "O",  "W", "O"]
    )

    ## Create basic rhythm map
    if beat_remainder == time_signature[2]
        for i in 1:time_signature[2]
            push!(rhythm_map, rhythms[time_signature][end - (time_signature[2] - i)])
        end
            
        for i in 0:(size(original, 2) - 1-time_signature[2])
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    else
        for i in 0:(size(original, 2) - 1)
            push!(rhythm_map, rhythms[time_signature][(i % (time_signature[1]*time_signature[2]) + 1)])
        end
    end
    return rhythm_map
end

display(rhythmify((3, 2), data[:, 5:end]))

20-element Array{String,1}:
 "W"
 "O"
 "S"
 "O"
 "W"
 "O"
 "W"
 "O"
 "S"
 "O"
 "W"
 "O"
 "W"
 "O"
 "S"
 "O"
 "W"
 "O"
 "W"
 "O"

In [161]:

# Child : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function function_cost(child, rhythm_map)
    
    current_child = child[end][1:2]
    current_function = current_child[2]
    current_index = size(child, 1)
    previous_child = child[end-1][1:2]
    previous_function = previous_child[2]
    previous_index = size(child, 1) - 1
    
    # Check the integrity of the function sequence
    total_cost = 0
    #for i in max(size(child, 1) - 1, 2):size(child, 1)
    
    # /5 chords     
    # T5, S5, D5 requires same function type before and after WORKS
    if (previous_function == "T/5" || previous_function == "S/5" || previous_function == "D/5")
        if string(child[end-2][2][1]) != string(current_function[1])
            #println("T5, S5, D5 requires same function type before and after")
            total_cost += 1000 
        end   
    end

    ## Suspended chords
    
    # T64, D64 must come on a stronger beat than their resolution WORKS
    if ((previous_function == "T64" && current_function != "T54") || previous_function == "T54" || (previous_function == "D64" && current_function != "D54") 
                || previous_function == "D54") && (beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]])
        #println("T64, D64 must come on a stronger beat than their resolution")
        total_cost += 1000
    end

    # D64 is introduced from T or S WORKS
    if current_function == "D64" && !(string(previous_function[1]) == "T" || string(previous_function[1]) == "S")
        #println("D64 is introduced from T or S")
        total_cost += 1000
    end

    # D64 becomes D54 or D WORKS
    if previous_function == "D64" && !(current_function == "D54" || current_function == "D")
        #println("D64 becomes D54 or D")
        total_cost += 1000
    end

    # D54 becomes D WORKS
    if previous_function == "D54" && current_function != "D"
        #println("D54 becomes D")
        total_cost += 1000
    end

    # T64 is introduced from S or D7 WORKS
    if current_function == "T64" && !(string(previous_function[1]) == "T" || string(previous_function[1]) == "S")
        #println("T64 is introduced from S or D7")
        total_cost += 1000
    end

    # T64 becomes T54 or T WORKS
    if previous_function == "T64" && !(current_function == "T54" || current_function == "T")
        #println("T64 becomes T54 or T")
        total_cost += 1000
    end

    # T54 becomes T WORKS
    if previous_function == "T54" && (current_function != "T")
        #println("T54 becomes T")
        total_cost += 1000
    end
    
    if total_cost > 999
        return total_cost
    end

    ## Dominants
    # Dominants can only go to T, S3 
    if previous_function in Dominants && !(current_function == "T" || current_function == "S/3" || current_function in Dominants)
        #println("Dominants can only go to T, S3 or more complex D's")
        total_cost += 1000
    end

    # Dominants can only go to more complex D's
    if previous_function in Dominants && current_function in Dominants && dominant_order[previous_function] >= dominant_order[current_function]
        #println("Dominants can only go to more complex D's")
        total_cost += 1000
    end
    
    ## Mediants
    
    # Don't go from a submediant to its chief
    if (previous_function == "Ts" && string(current_function[1]) == "T") || (previous_function == "Tm" && string(current_function[1]) == "D") || 
        (previous_function == "Ss" && string(current_function[1]) == "S") || (previous_function == "Tm" && string(current_function[1]) == "T")
        total_cost += 1000
    end
    
    # Ts -> S/3 or Ss -> D7-1 must decrease in rhythmic accentuation
    if ((previous_function == "Ts" && current_function == "S/3") || (previous_function == "Ss" && current_function == "D7-1")) && 
        beat_accentuation[rhythm_map[previous_index]] < beat_accentuation[rhythm_map[current_index]]
        total_cost += 1000
    end
    
    if i > 2 && (previous_function == "Tm" || previous_function == "Ts" || previous_function == "Dm")
        ancient_child = child[end-2][1:2]
        ancient_function = previous_child[2]
        ancient_index = size(child, 1) - 2 
        
        if (string(ancient_function[1]) == "T" && previous_function == "Tm" && string(current_function[1]) == "D")
            total_cost += 1000
        end 
    end
    
    # Avoid repeating /3 or /5 chords
    
    if (current_function == "T/3" && previous_function == "T/3") || (current_function == "S/3" && previous_function == "S/3") || (current_function == "D/3" && previous_function == "D/3") ||
        (current_function == "T/5" && previous_function == "T/5") || (current_function == "S/5" && previous_function == "S/5") || (current_function == "D/5" && previous_function == "D/5")
        total_cost += 1000
    end
    
    
    
    if total_cost > 999
        return total_cost
    end
    
    mediant_counter = 0
    
    # Full check of chord repetition
    for i in 1:size(child, 1)
        
        total_cost += child[i][1]
        
        if i in size(child, 1)-1:size(child, 1) && i != 1
            
            
        
            current_child = child[i][1:2]
            #println("current_child ", current_child)
            current_function = current_child[2]
            #println("current_func ", current_function)
            current_index = i
            previous_child = child[i-1][1:2]
            #println("previous_child ", previous_child)
            previous_function = previous_child[2]
            #println("previous_child ", previous_function)
            previous_index = i-1
        
        
            # Chord repetition can only happen from stronger to weaker beats. Otherwise, cost is set by distance to original tonal_map.
            if (previous_function == current_function) && !(rhythm_map[i] == "A" || rhythm_map[i-1] == "A")
                if (beat_accentuation[rhythm_map[i]] >= beat_accentuation[rhythm_map[i-1]])
                    total_cost += 1000
                end
            end
        end
        
        if child[i][2] in ["Ts", "Tm", "Ss"]
            mediant_counter += 1
        end
                
    end
    
    # Max 1 mediant per bar
    if mediant_counter > size(rhythm_map, 1) / (time_signature[1] * time_signature[2])
        total_cost += 1000
    end
    
    return total_cost
end

hidden_parallell =  [(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), 
                    (8, "S65", [(53, 1), (60, 5), (62, 6), (69, 3)], 13, 5), 
                    (9, "D7/7", [(53, 7), (62, 5), (67, 1), (71, 3)], 22, 12), 
                    (5, "T", [(48, 1), (55, 5), (64, 3), (72, 1)], 27, 25), 
                    (8, "D/3", [(47, 3), (62, 5), (67, 1), (79, 1)], 35, 41), 
                    (8, "D7/7", [(53, 7), (62, 5), (71, 3), (79, 1)], 43, 64), 
                    (3, "S/3", [(57, 3), (60, 5), (65, 1), (77, 1)], 46, 93), 
                    (12, "Tm", [(52, 1), (59, 5), (67, 3), (79, 3)], 58, 126), 
                    (5, "S65/6", [(50, 6), (60, 5), (69, 3), (77, 1)], 63, 159), 
                    (11, "D7/7", [(53, 7), (62, 5), (67, 1), (71, 3)], 74, 197), 
                    (4, "T", [(48, 1), (55, 5), (64, 3), (72, 1)], 78, 243), 
                    (6, "T/3", [(52, 3), (55, 5), (67, 5), (72, 1)], 84, 294), 
                    (11, "D", [(55, 1), (55, 1), (62, 5), (71, 3)], 95, 350), 
                    (3, "D7", [(55, 1), (62, 5), (65, 7), (71, 3)], 98, 409), 
                    (5, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 103, 472), 
                    (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 106, 534)]

rhythm_map = ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
                
println(function_cost(hidden_parallell, rhythm_map))

106


In [180]:

## Function for calculating the voicing cost between neighbouring voicings
# Child : [1:(1:function1_cost, 2:"function1_name", 3:[function1_voicing_list]), 2:(1:function2_cost, 2:"function2_name", , 3:[function2_voicing_list]), ...]
function voicing_cost(child, rhythm_map_local, full=false, debug=false)
    
    
    if full
        total_cost = 0
        from = 2
    else
        from = size(child, 1)
        total_cost = child[end-1][5]
    end
    
        
    
    
    for i in from:size(child, 1)
            
        
        current_notes = child[i][3]
        current_function = child[i][2]
        previous_notes = child[i-1][3]
        previous_function = child[i-1][2]
        
            
        dist = [current_notes[1][1] - previous_notes[1][1], current_notes[2][1] - previous_notes[2][1], current_notes[3][1] - previous_notes[3][1], current_notes[4][1] - previous_notes[4][1]]
            
        if i > 2
            ancient_notes = child[i-2][3]
            ancient_function = child[i-2][2]
            previous_dist = [previous_notes[1][1] - ancient_notes[1][1], previous_notes[2][1] - ancient_notes[2][1], previous_notes[3][1] - ancient_notes[3][1], previous_notes[4][1] - ancient_notes[4][1]]
               
            # bass stuff
            if abs(previous_dist[1]) > 2 && !(sign(previous_dist[1]) != sign(dist[1])) && abs(dist[1]) > 2
                if debug
                    println("Leaps in bass should be countered by a move in the other direction")
                end
                total_cost += 2              
            elseif (sign(previous_dist[1]) != sign(dist[1]))
                if debug
                    println("Encourage walking bass")
                end
                total_cost += 1              
            end
                    
            if abs(previous_dist[4]) > 2 && abs(dist[1]) > 2
                if debug
                    println("Leaps in bass should be countered by a move in the other direction")
                end
                total_cost += 2              
            end
                
        end

        ### Voice specific rules
        
        # Avalanche check

        if abs(reduce(+, map(x -> sign(x), dist))) == 4
            if rhythm_map_local[i] == "A" && reduce(+, map(x -> sign(x), dist)) == -4 && (abs(dist[4]) < 3)
                continue
            else
                if debug
                    println("Avalanche")
                end
                total_cost += 1000
            end
        end
          
        # Immediately return if cost > 1000     
        if total_cost > 999 && !debug
            return total_cost 
        end

        # voice movement cost = sum of all voices movement IF it exceeds 2 (step-wise motion has no cost)

        #total_cost += reduce(+, map(x -> abs(x) > 2 ? abs(x)-2 : 0 , distance)) # Penalize jumps depending on distance
        total_cost += reduce(+, map(x -> abs(x) > 2 ? 1 : 0 , dist)) # Flat 1 penalty for leaps

            
        # distance from original cost

        # TODO: Might mess things up

            
        # Prevent chord repetition

        if (reduce(+, map(x -> abs(x), dist)) == 0)
                    
            if rhythm_map_local[i] != "A"
                if debug
                    println("Repetition")
                end
                total_cost += 1000
            else
                total_cost -= 2
            end
        end
               
        # MAYBE: Encourage opposite movement between bass and soprano
        #if sign(dist[1]) == sign(dist[4]) && dist[1] != 0
            #total_cost += 4
        #end
            
        # Bass should be encouraged to move step-wise
        if abs(dist[1]) > 2
            total_cost += 2         
        end
                    
        # Mid voices must move < 8
        if !(dist[2] < 8) || !(dist[3] < 8)
            if debug
                println("Mid voices must move < 8")
            end
            total_cost += 1000
        end
        

        # Outer voices should only move < 8
        if !(abs(dist[1]) < 8) || !(abs(dist[4]) < 8)
            total_cost += 1
        end

            
        # Outer voices may move < 9 or 12 at a cost
        if (abs(dist[1]) >= 8 && abs(dist[1]) != 12) || (abs(dist[1]) > 7 && abs(dist[1]) < 10 && previous_function != current_function) || (abs(dist[4]) >= 10 && abs(dist[4]) != 12)
            if debug
                println("Outer voices may move < 9 or 12 at a cost")
            end
            total_cost += 1000
        end

        # Parallell primes, fifths and octaves are illegal WORKS
        if current_notes != previous_notes
            pairs = [(1, 2), (2, 3), (3, 4), (1, 3), (2, 4), (1, 4)]
            for (voice1, voice2) in pairs
                if abs(current_notes[voice1][1] - current_notes[voice2][1]) == 0 &&  abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 0
                    if debug
                        println("Parallell prime")
                    end
                    total_cost += 1000
                elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) == 7 &&  abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 7
                    total_cost += 1000
                    if debug
                        println("Parallell fifth")
                    end
                elseif abs(current_notes[voice1][1] - current_notes[voice2][1]) == 12 &&  abs(previous_notes[voice1][1] - previous_notes[voice2][1]) % 12 == 12
                    total_cost += 1000 
                    if debug
                        println("Parallell octave")
                    end
                end
            end
        end

        
        # Hidden parallell fifths and octaves are illegal WORKS!
        if (sign(dist[1]) == sign(dist[4])) && abs(dist[4]) > 2 && ((abs(current_notes[1][1] - current_notes[4][1]) % 12 == 0) || (abs(current_notes[1][1] - current_notes[4][1]) % 12 == 7))
            if debug
                println("Hidden parallell fifths and octaves are illegal")
            end
            total_cost += 1000
        end
                        


        ### Chord specific rules

        ## Dominants               
        
        # From S to D, bass moves stepwise up while rest move shortest path down
        if (previous_function == "S" && current_function == "D") && dist[1] != 2 && rhythm_map_local[i] != "A"
            if debug
                    println("From S to D, bass moves stepwise up while rest move shortest path down")
                end
            total_cost += 1000 
        end
                        

        if total_cost > 999 && !debug
            return total_cost 
        end
                        
        # Leading note in D goes to root, except can leap down a 3rd to 5 in mid voices # WORKS ?           
        if string(previous_function[1]) == "D" && string(current_function[1]) == "T"
               
            for (j, voice) in enumerate(previous_notes)
                                
                if rhythm_map_local[i] == "A" 
                    if voice[2] == 3 && !((current_notes[j][2] == 1 && dist[j] == 1) || (current_notes[j][2] == 5 && dist[j] == -4)) && !(j == 2 || j == 3)
                        total_cost += 1000
                    end
                elseif voice[2] == 3 && current_notes[j][2] != 1 && dist[j] != 1
                    total_cost += 1000
                end
            end
                            
        end
                            
                            
        ## /5 chords WORKS
                        
        # Bass and voice with /5 root must move max step-wise in and out.
        if current_function == "T/5" || current_function == "D/5" || current_function == "S/5" 
            if dist[1] > 2 
                total_cost += 1000   
            end
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 1 && dist[j] > 2
                    total_cost += 1000
                end
            end
        end
                        
        if previous_function == "T/5" || previous_function == "D/5" || previous_function == "S/5"
            if dist[1] > 2 
                total_cost += 1000   
            end
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 1 && dist[j] > 2
                    total_cost += 1000
                end
            end
        end
        
        ## Augmented chords
                        
        # The 4 in 64 chords must be introduced stationary and resolved step-wise                
        if current_function == "T64" || current_function == "D64"
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 4 && dist[j] != 0
                    if debug
                        println("4 is not stationary in augmented")
                    end
                    total_cost += 1000
                end
                if voice[2] == 6 && abs(dist[j]) > 2
                    if debug
                        println("6 leaps in augmented")
                    end
                    total_cost += 2
                end
            end
        end
                        
        if previous_function == "T64" || previous_function == "D64"
            if dist[1] != 0
                total_cost += 1000   
            end
                            
            if string(current_function[1]) == string(ancient_function[1])
                total_cost += 1000
            end
                                
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 4 && (dist[j] < -2 || dist[j] > 0)
                    if debug
                        println("4 is not resolved step-wise down")
                    end
                    total_cost += 1000
                end
                if voice[2] == 6 && (dist[j] < -2 || dist[j] > 0)
                    if debug
                        println("6 is not resolved step-wise down")
                    end
                    total_cost += 1000
                end
            end
        end
        
        # The 4 in T54 must be introduced stationary and resolved step-wise down
        if current_function == "T54" || current_function == "D54"
                            
            for (j, voice) in enumerate(current_notes)
                if voice[2] == 4 && dist[j] != 0
                    if debug
                        println("4 is not stationary in augmented")
                    end
                    total_cost += 1000
                end
            end
        end
         
        if previous_function == "T54" || previous_function == "D54"
            if dist[1] != 0
                total_cost += 1000   
            end
                            
            if string(current_function[1]) == string(ancient_function[1])
                total_cost += 1000
            end
                            
            for (j, voice) in enumerate(previous_notes)
                if voice[2] == 4 && (dist[j] < -2 || dist[j] > 0)
                    if debug
                        println("4 is not resolved step-wise down")
                    end
                    total_cost += 1000
                end
            end
        end
                                
                            
        ## /3 chords
                            
        # /3 chords together are encouraged, but only if correct voicing is used
        if i > 2 && (previous_function == "T/3" || previous_function == "S/3" || previous_function == "D/3")
            if debug
                println("S/3 Error")
            end
            if (ancient_function == "T/3" && previous_function == "D/3") || (ancient_function == "S/3" && previous_function == "T/3") || (ancient_function == "T/3" && previous_function == "S/3")
                total_cost -= 1
                                
                # Leaps in bass should be countered by a move in the other direction
                if (abs(previous_dist[1]) > 2) && !(sign(previous_dist[1]) != sign(dist[1])) || !(sign(previous_dist[1]) != sign(previous_dist[4]))
                    if debug
                        println("S/3 Leaps in bass should be countered by a move in the other direction")
                    end
                    total_cost += 1000              
                end
                                
            elseif (ancient_function == "T" && previous_function == "D/3" && current_function == "T") || (ancient_function == "S" && previous_function == "T/3" && current_function == "S") ||
                                    (ancient_function == "D" && previous_function == "T/3" && current_function == "D") || 
                                    (ancient_function == "T" && previous_function == "S/3" && current_function == "T") ||
                                    (ancient_function == "D" && previous_function == "T/3" && current_function == "S") || 
                                    (ancient_function == "S" && previous_function == "T/3" && current_function == "D")
                                    
                total_cost -= 1
                                    
            elseif (ancient_function == "T" && previous_function == "T/3" && current_function == "S") || (ancient_function == "T" && previous_function == "T/3" && current_function == "D") ||
                                    (ancient_function == "S" && previous_function == "S/3" && current_function == "T") || (ancient_function == "D" && previous_function == "D/3" && current_function == "T-5") ||
                                    (ancient_function == "D" && previous_function == "D/3" && current_function == "T") || (ancient_function == "T" && previous_function == "D/3" && current_function == "D") ||
                                    (ancient_function == "T" && previous_function == "S/3" && current_function == "S") || (ancient_function == "S" && previous_function == "T/3" && current_function == "T")
                    
                total_cost -= 1                     
            end
        end
                          
                                    
        # TODO
    
    end
    
    return total_cost
        
end
            
#println(voicing_cost((0, "T", [()])), ["S", "W"])

#println(sign(48 - 60) == sign(72 - 77) && (abs(-5) > 2) && ((abs(48 - 72) % 12 == 0) || (abs(48 - 72) % 12 == 7)))
                
 #previous [(60, 1), (60, 1), (67, 5), (77, 4)]
 #current [(48, 1), (64, 3), (67, 5), (72, 1)]
                
                
hidden_parallell =  [(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 5, 4), (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 6, 14), (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 8, 30), (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 8, 54), (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 11, 81), (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 13, 118), (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 17, 156), (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 19, 198), (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 20, 244), (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 23, 294), (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 26, 353), (6, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 32, 416), (7, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 39, 480), (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 44, 548), (3, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 47, 620)]

rhythm_map = ["A", "A", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]
                
println(voicing_cost(hidden_parallell, rhythm_map, false, true))
                
#dist = [0, 0, 0, 0]
#println((reduce(+, map(x -> abs(x), dist)) == 0) && "S" != "T")
                



Encourage walking bass
552


In [243]:
# Rate the voicing permutation with regards to the original score

function distance_cost(child, timestep, debug=false)
    
    total_dist = 0
    voices = child[end][3]
    active_notes = getActiveNotes(timestep)
    
    println(active_notes)
    
    for (i, voice) in enumerate(voices)
        
        if i == 1
            total_dist += abs(active_notes[1] - voice[1])
            println(abs(active_notes[1] - voice[1]))
        elseif i == 2 || i == 3
            println([abs(active_note - voice[1]) for active_note in active_notes])
            total_dist += minimum([abs(active_note - voice[1]) for active_note in active_notes])
        elseif i == 4
            total_dist += abs(active_notes[end] - voice[1])
            println(abs(active_notes[end] - voice[1]))
        end
    end
                
    println(total_dist)
            
    return total_dist
 
end

distance_cost([(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)], 1, true)

[60, 64, 74, 85]
7
[5, 1, 9, 20]
[9, 5, 5, 16]
11
24


# # DFS Branch and Bound ALL IN ONE

In [197]:

function prepare(data, functions, function_potential)
    
    Tonics = ["T", "T/3", "T/5", "T-5"]
    Tonics_dependant = ["T64", "T54"]
    Subdominants = ["S", "S/3", "S/5", "S65", "S65/6", "S6"]
    Dominants = ["D", "D/3", "D/5", "D-5", "D64", "D54", "D7", "D7-1", "D7-5", "D7/3", "D7/5", "D7/7"]
    Mediants = ["Ts", "Tm", "Ss", "Dm"]
    beat_accentuation = Dict("O" => 1, "W" => 2, "S" => 3)
    dominant_order = Dict("D" => 3, "D/3" => 3, "D/5" => 3, "D-5" => 1, "D64" => 1, "D54" => 2, "D7" => 5, "D7-1" => 4, "D7-5" => 4, "D7/3" => 5, "D7/5" => 5, "D7/7" => 5)
    
    start = ["T"]
    leading_end = ["D", "D7"]
    final = ["T"]
    
    # Local copy of the original data
    data_local = deepcopy(data)
    # Local copy of dictionary with global function rules : dict[key] = ([1: tonal_map], [2: bass note], [3: double/triple note], [4: Rhythmical rules])
    functions_local = deepcopy(functions)  
    
    # Local copy of list of candidate functions for each timestep [[timestep1], [timestep2], ... ]
    # Each index contains a sorted list of tuples which determine function costs for the given timestep : function_potential_local[i] = [(function_cost, "function_name"), ...]
    function_potential_local = deepcopy(function_potential[1:size(data_local, 2)])
    function_permutations_local = deepcopy(function_permutations)
    
    
    # Create rhythm map for piece
    rhythm_map_local = rhythmify(time_signature, data)
    
    
    # Locate and enforce important events (start, cadences)
    
    start_index = findfirst(x -> x == "S", rhythm_map_local)
    final_index = findlast(x -> x == "S" || x == "W", rhythm_map_local)
    leading_index = findlast(x -> x == "W" || x == "S", rhythm_map_local[1:final_index-1])
    
    filter!(x -> x[2] in start, function_potential_local[start_index])
    for i in leading_index:final_index-1
        filter!(x -> x[2] in Dominants, function_potential_local[i])
        #rhythm_map_local[i] = "D"
    end
    
    filter!(x -> x[2] in leading_end, function_potential_local[final_index-1])
    #rhythm_map_local[final_index-1] = "D"
    for i in final_index:size(data, 2)
        filter!(x -> x[2] in final, function_potential_local[i])
        rhythm_map_local[i] = "A" # Authentic cadence
    end
    
    println("Rhythm map: ", rhythm_map_local)
    
    
    # Prune function_potential by beat restrictions for each chord
    
    for (i, potential) in enumerate(function_potential_local)
        for (j, (cost, chord)) in enumerate(potential)
            if !(rhythm_map_local[i] in functions_local[chord][4])
                filter!(x -> x[2] != chord, potential)
            end
        end
    end
    
    #display(function_potential_local)
    
    initial_solution, bestval_func, bestval_voices = BB(function_potential_local, rhythm_map_local, data_local, function_permutations_local, 500, 500, true)
    println("Initial solution found")
    display(initial_solution)
    println("Initial best function score: ", bestval_func, ", best voices score: ", bestval_voices)
    
    best_solution = BB(function_potential_local, rhythm_map_local, data_local, function_permutations_local, bestval_func, bestval_voices)

    return best_solution
end

function BB(function_potential_local, rhythm_map_local, data_local, function_permutations_local, bestval_func, bestval_voices, exploratory=false)
        
    # Initialize global search variables
    best_solution = []
    best_score = 0
    total_depth = size(data_local, 2)
    
    # Value larger than or equal to the largest value in set
    upper_bound = 1000
    # Value smaller than or equal to the smallest value in set
    lower_bound = 0
    
    #display(function_potential_local)
    
    min_list = [timestep[1][1] for timestep in function_potential_local]
    println(min_list)
    
    
    # Populate active set with all permutations of the possible first functions
    activeset = Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1}, Int64, Int64},1}[]

    # Func : (cost, function)
    for (cost, func) in function_potential_local[1]
        for potential_voicing in function_permutations_local[func]
            temp = [(cost, func, potential_voicing, 0, 0)]
            push!(activeset, [(cost, func, potential_voicing, 0, 0)])
        end
    end
    display(activeset)
    
    bestval_func = bestval_func
    bestval_voices = bestval_voices
    solution_list = []
    
    ###### TestZone
    
    #function_potential_local[5] = [(0, "D")]
    #function_potential_local[6] = [(0, "S/3")]
    
    ######
    
    iterations = 0
    pruned = 0
    processed = 0
    solutions = 0
    last_solution = 0

    
    println("Starting search...")
    
    start = time()
    
    while !isempty(activeset)
        iterations += 1

        active_node = pop!(activeset)
            
        depth = size(active_node, 1)
    
        
        # Generate children of active_node and corresponding optimistic bounds
        #for (cost, func) in Iterators.reverse(deepcopy(function_potential_local[depth+1]))
        for (cost, func) in Iterators.reverse(function_potential_local[depth+1]) #
            #println(cost, func)
            
            if depth == total_depth/2
                println("Visiting middle")
            end
            
            # Child : [(function1_cost, "function1_name", [function1_voicing_list]), (function2_cost, "function2_name", , [function2_voicing_list]), ...]
            for voicing in function_permutations_local[func]
                
                child = deepcopy(active_node) 
                #child = active_node
                temp_child = (cost, func, voicing, 0, 0)
                push!(child, temp_child)
                processed += 1
                
                child_function_cost = function_cost(child, rhythm_map_local)*2
                child_function_heuristic = sum(min_list[size(child, 1)+1:end]) 
                child_voicing_heuristic = 2*(total_depth - size(child, 1))
                
                child_voicing_cost = voicing_cost(child, rhythm_map_local, false, false)
                child_total_cost = child_function_cost + child_function_heuristic + child_voicing_cost + child_voicing_heuristic
                
                temp_child = (cost, func, voicing, child_function_cost, child_voicing_cost)
                deleteat!(child, size(child, 1))
                push!(child, temp_child)
                
                ## TODO: add cost for poor voicing permutation matchings
                
                # DEBUG
                
                if child_total_cost > bestval_func + bestval_voices 
                    
                    deleteat!(child, size(child, 1))
                    pruned += 1     
            
                elseif size(child, 1) == total_depth
                    
                    solutions += 1
                    #child_voicing_cost = voicing_cost(child, rhythm_map_local, true)
                    #child_total_cost = child_function_cost + child_function_heuristic + child_voicing_cost
                    println("Child voicing heuristic for done: ", child_voicing_heuristic)
                        
                    if exploratory == true
                        solution = child
                        bestval_func = child_function_cost
                        bestval_voices = child_voicing_cost
                        return solution, bestval_func, bestval_voices

                    elseif child_total_cost <= bestval_func + bestval_voices
                        
                        last_solution = iterations
                        bestval_func = child_function_cost
                        bestval_voices = child_voicing_cost
                        println("Current child: ", child)
                        println("function cost: ", child_function_cost + child_function_heuristic)
                        println("child voicing cost: ", child_voicing_cost + child_voicing_heuristic)
                        println("Best yet: ", bestval_func + bestval_voices)
                            
                        if child_total_cost == bestval_func + bestval_voices
                            push!(solution_list, (child_total_cost, child))
                                
                        elseif child_total_cost < bestval_func + bestval_voices
                            bestval_func = child_function_cost
                            bestval_voices = child_voicing_cost
                            solution_list = [(child_total_cost, child)]
                                    
                        end
                    end
                else
                    push!(activeset, child)
                end
            end
            #println(activeset)
        end
        
        if iterations % 1000 == 0
            println(size(activeset, 1))
            println(iterations)
        end     
              
        if exploratory && iterations == 1000
            return "No initial solution found"
        elseif iterations == 100000 #|| iterations - last_solution == 10000
            println("Timed out")
            display(activeset)
            break
        end
                        
        ## Maybe add automatic termination if either within 1+a approx of optimistic minimum, after x iterations of no new solutions found, or when crossing a certain efficiency threshold
    end
                
    elapsed = time() - start
    println("Done. Total checked: ", processed, ", Pruned: ", pruned, ", completed: ", solutions, ", time: ", elapsed, ", efficiency: ", (solutions)/elapsed)
    println("Results: bestval_func: ", bestval_func, ", bestval_voices: ", bestval_voices)
    return solution_list
end

solution_list = prepare(data[:, 1:16], functions, function_potential)
            


Rhythm map: ["S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "W", "O", "S", "O", "A", "A"]


15-element Array{Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1},1}:
 [(5, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 0, 0)]
 [(5, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 0, 0)]
 [(5, "T", [(60, 1), (64, 3), (67, 5), (72, 1)], 0, 0)]
 [(5, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 0, 0)]
 [(5, "T", [(48, 1), (64, 3), (72, 1), (79, 5)], 0, 0)]
 [(5, "T", [(60, 1), (64, 3), (72, 1), (79, 5)], 0, 0)]
 [(5, "T", [(48, 1), (55, 5), (64, 3), (72, 1)], 0, 0)]
 [(5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 0, 0)]
 [(5, "T", [(48, 1), (67, 5), (72, 1), (76, 3)], 0, 0)]
 [(5, "T", [(60, 1), (67, 5), (72, 1), (76, 3)], 0, 0)]
 [(5, "T", [(48, 1), (60, 1), (64, 3), (67, 5)], 0, 0)]
 [(5, "T", [(60, 1), (60, 1), (64, 3), (67, 5)], 0, 0)]
 [(5, "T", [(48, 1), (48, 1), (55, 5), (64, 3)], 0, 0)]
 [(5, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 0, 0)]
 [(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)]

[5, 0, 1, 2, 0, 1, 2, 0, 2, 1, 2, 2, 2, 3, 5, 3]
Starting search...
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle


16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46)  
 (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50)
 (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59)  
 (6, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 64, 63)
 (7, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 78, 64)  
 (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 68)  
 (3, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 9

15-element Array{Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1},1}:
 [(5, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 0, 0)]
 [(5, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 0, 0)]
 [(5, "T", [(60, 1), (64, 3), (67, 5), (72, 1)], 0, 0)]
 [(5, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 0, 0)]
 [(5, "T", [(48, 1), (64, 3), (72, 1), (79, 5)], 0, 0)]
 [(5, "T", [(60, 1), (64, 3), (72, 1), (79, 5)], 0, 0)]
 [(5, "T", [(48, 1), (55, 5), (64, 3), (72, 1)], 0, 0)]
 [(5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 0, 0)]
 [(5, "T", [(48, 1), (67, 5), (72, 1), (76, 3)], 0, 0)]
 [(5, "T", [(60, 1), (67, 5), (72, 1), (76, 3)], 0, 0)]
 [(5, "T", [(48, 1), (60, 1), (64, 3), (67, 5)], 0, 0)]
 [(5, "T", [(60, 1), (60, 1), (64, 3), (67, 5)], 0, 0)]
 [(5, "T", [(48, 1), (48, 1), (55, 5), (64, 3)], 0, 0)]
 [(5, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 0, 0)]
 [(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)]

Child voicing heuristic for done: 0
Initial solution found
Initial best function score: 94, best voices score: 72
[5, 0, 1, 2, 0, 1, 2, 0, 2, 1, 2, 2, 2, 3, 5, 3]
Starting search...
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Visiting middle
Child voicing heuristic for done: 0
Current child: Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64}[(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4), (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10), (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16), (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24), (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27), (2,

819-element Array{Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1},1}:
 [(5, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 0, 0)]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
 [(5, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 0, 0)]                                                                                                               

819
100000
Timed out
Done. Total checked: 4135431, Pruned: 4034590, completed: 37, time: 68.74300003051758, efficiency: 0.5382366202169577
Results: bestval_func: 94, bestval_voices: 52


37-element Array{Any,1}:
 (166, Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64}[(5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0), (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4), (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10), (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16), (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24), (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27), (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37), (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38), (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42), (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46), (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50), (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59), (6, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 64, 63), (7, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 78, 64), (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 68), (3, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 94, 72)]) 
 (

# # Collect solution voicings and create piano roll

In [198]:
for solution in solution_list
    display(solution[2])
end

solution_notes = [timestep[3] for timestep in solution_list[end][2]]
solution_functions = [timestep[2] for timestep in solution_list[end][2]]
display(solution_functions)
println(solution_functions[end])

## Convert back to piano roll

function makePianoRoll(notes)
    println(size(notes, 1))
    pianoroll = zeros(Int8, 128, size(notes,1))
    counter = 1
    for set in notes[:]
        for voice in set
            #println(voice)
            pianoroll[voice[1], counter] = 1
        end
        counter = counter + 1
        #println(pianoroll[:,1])
    end
    #println(findall(x->x==1, pianoroll))
    empty = zeros(Int8, 1, size(notes, 1))
    new = vcat(pianoroll, empty)
    display(new)
    writedlm("output\\test.csv",  new, ',')
end

makePianoRoll(solution_notes)




16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46)  
 (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50)
 (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59)  
 (6, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 64, 63)
 (7, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 78, 64)  
 (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 68)  
 (3, "T", [(48, 1), (52, 3), (55, 5), (60, 1)], 9

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46)  
 (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50)
 (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59)  
 (6, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 64, 63)
 (7, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 78, 64)  
 (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 68)  
 (3, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 9

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46)  
 (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50)
 (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59)  
 (6, "D64", [(43, 1), (55, 1), (60, 4), (64, 6)], 64, 63)
 (7, "D", [(43, 1), (55, 1), (59, 3), (62, 5)], 78, 64)  
 (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 68)  
 (3, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 9

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46)  
 (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50)
 (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59)  
 (6, "D64", [(43, 1), (52, 6), (60, 4), (67, 1)], 64, 60)
 (7, "D", [(43, 1), (50, 5), (59, 3), (67, 1)], 78, 61)  
 (5, "T", [(48, 1), (48, 1), (55, 5), (64, 3)], 88, 67)  
 (3, "T", [(48, 1), (48, 1), (55, 5), (64, 3)], 9

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 40, 46)  
 (3, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 46, 50)
 (3, "S", [(41, 1), (53, 1), (60, 5), (69, 3)], 52, 59)  
 (6, "D64", [(43, 1), (52, 6), (60, 4), (67, 1)], 64, 60)
 (7, "D", [(43, 1), (50, 5), (59, 3), (67, 1)], 78, 61)  
 (5, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 88, 65)  
 (3, "T", [(48, 1), (52, 3), (60, 1), (67, 5)], 9

...

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D/3", [(47, 3), (55, 1), (55, 1), (62, 5)], 40, 45)
 (4, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 48, 46)  
 (3, "S", [(53, 1), (57, 3), (60, 5), (65, 1)], 54, 49)  
 (9, "D-5", [(55, 1), (55, 1), (59, 3), (67, 1)], 72, 49)
 (3, "D7", [(55, 1), (59, 3), (62, 5), (65, 7)], 78, 52) 
 (5, "T", [(48, 1), (48, 1), (55, 5), (64, 3)], 88, 53)  
 (3, "T", [(48, 1), (48, 1), (55, 5), (64, 3)], 9

16-element Array{Tuple{Int64,String,Array{Tuple{Int64,Int64},1},Int64,Int64},1}:
 (5, "T", [(60, 1), (60, 1), (67, 5), (76, 3)], 0, 0)    
 (0, "S6", [(53, 1), (65, 1), (69, 3), (74, 6)], 10, 4)  
 (1, "T", [(48, 1), (60, 1), (67, 5), (76, 3)], 12, 10)  
 (2, "S", [(53, 1), (65, 1), (69, 3), (72, 5)], 16, 16)  
 (0, "T54", [(48, 1), (65, 4), (72, 1), (79, 5)], 16, 24)
 (3, "T", [(48, 1), (64, 3), (67, 5), (72, 1)], 22, 27)  
 (2, "D64", [(43, 1), (55, 1), (64, 6), (72, 4)], 26, 37)
 (4, "D", [(43, 1), (55, 1), (62, 5), (71, 3)], 34, 38)  
 (2, "S/3", [(45, 3), (60, 5), (65, 1), (65, 1)], 38, 42)
 (1, "D/3", [(47, 3), (55, 1), (55, 1), (62, 5)], 40, 45)
 (4, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 48, 46)  
 (3, "S", [(53, 1), (57, 3), (60, 5), (65, 1)], 54, 49)  
 (9, "D-5", [(55, 1), (55, 1), (59, 3), (67, 1)], 72, 49)
 (3, "D7", [(55, 1), (59, 3), (62, 5), (65, 7)], 78, 52) 
 (5, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 88, 53)  
 (3, "T", [(48, 1), (55, 5), (60, 1), (64, 3)], 9

16-element Array{String,1}:
 "T"  
 "S6" 
 "T"  
 "S"  
 "T54"
 "T"  
 "D64"
 "D"  
 "S/3"
 "D/3"
 "T"  
 "S"  
 "D-5"
 "D7" 
 "T"  
 "T"  

129×16 Array{Int8,2}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮              ⋮
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0 

T
16


In [15]:
display(solution_notes)

UndefVarError: UndefVarError: solution_notes not defined

In [16]:
empty = zeros(Int8, 1, size(data, 2))
input_fixed = deepcopy(data)
new = vcat(input_fixed, empty)
writedlm("input\\dummyMajorMultiple.csv",  new, ',')

# # Construct a naive solution

# # NODAL

In [17]:
import Optim
SimulatedAnnealing(; neighbor = default_neighbor!,
                    T = default_temperature,
                    p = kirkpatrick)

UndefVarError: UndefVarError: default_neighbor! not defined